# Except for some extremely minor revisions, all of the credit for this notebook goes to Matteo Esposito!

In [1]:
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import copy
import camb
from utilities import *
%load_ext autoreload
%autoreload 2
#Plotting settings
%matplotlib inline
plt.rcParams.update({'axes.labelsize': 'x-large'})
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
linestyles=['solid', '--', '-.', ':', (0, (3, 5, 1, 5, 1, 5)), (0, (1, 10))]
colors=list(mcolors.TABLEAU_COLORS.values())#['orange', 'blue', 'green', 'magenta', 'red', 'cyan']

cosmo_dict = make_cosmo_suite(fname='/home/lfinkbei/Documents/Master/' + \
    'CAMB_notebooks/shared/data/cosmologies.dat', use_sigma8=False)

2023-04-04 14:48:23.875176: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model0 = {
    'ombh2' : 0.022445,
    'omch2' : 0.120567,
    'n_s' : 0.96,
    'A_s' : 2.12723788013000E-09,
    'h' : 0.67,
    'w0' : -1.00,
    'wa' : 0.00,
    'OmK' : 0.00
}            

In [28]:
def get_camb_Pk(mlc, mnu=0., sim_idx=0, z_idx=range(5), z=None, ns = 0.96):
    '''
    print("s12", s12)
    print("sim_idx", sim_idx)
    '''
    
    #Set up a new set of parameters for CAMB
    try:
        # Lukas. This line is obscure.
        zz = [1/cosmo_dict['Columbus_%d_%d' %(sim_idx, i)]['expfactor']-1 for i in z_idx]
        z_idx = z_idx[0]
    except:
        pass
    
    if z is not None:
        zz = z

    cosmo_params = cosmo_dict['Columbus_%d_%d' %(sim_idx, z_idx)]
    pars = camb.CAMBparams()
    h = mlc['h']
    ombh2 = mlc['ombh2']
    omch2 = mlc['omch2']
    omk = mlc['OmK']
   
    pars.set_cosmology(H0=h*100, ombh2=ombh2,mnu=mnu, omk=omk,
        tau=0.0952, neutrino_hierarchy="degenerate", omch2=omch2)
    pars.set_cosmology(H0=h*100, ombh2=ombh2, mnu=mnu, omk=omk,
        tau=0.0952, neutrino_hierarchy="degenerate", omch2=omch2-pars.omnuh2)
    
    ''' Lukas
    I'm commenting out these definitions because they are unused.
    fb = pars.ombh2/(pars.ombh2+pars.omch2+pars.omnuh2)
    fnu = pars.omnuh2/(pars.ombh2+pars.omch2+pars.omnuh2)
    '''
    
    pars.InitPower.set_params(As=mlc['A_s'], ns=mlc['n_s'],
        r=0, nt=0.0, ntrun=0.0)
    pars.NonLinear = camb.model.NonLinear_none
    pars.WantCls = False
    pars.WantScalars = False
    pars.Want_CMB = False
    pars.DoLensing = False
    pars.YHe = 0.24
    pars.set_accuracy(AccuracyBoost=2)
    if mlc['w0'] != -1 or mlc['wa'] != 0:
        pars.set_dark_energy(w=w0, wa=wa, dark_energy_model='ppf') 
        
    # Lukas. I added the "/ h"
    # Lukas. I've verified that for model 0, these redshifts are correct.
    pars.set_matter_power(redshifts=zz, kmax=20.0 / h, nonlinear=False)    
        
    results = camb.get_results(pars)

    R, zz, sigma12 = results.get_sigmaR(12, hubble_units=False, return_R_z=True)
    
    # Lukas: I changed var1 and var2 to 8, they were both 2 before
    # Lukas: I changed npoints to 100000, it was 200 before
    kh, z, pkh = results.get_matter_power_spectrum(var1=8, var2=8, minkh=1e-4/h,
        maxkh=10.0 / h, npoints = 100000) 
    
    return {'kk': kh*h, 'zz': z, 'Pk': pkh/(h**3)}

In [29]:
nsims = 9
LCDM = get_camb_Pk(model0, mnu=0)

mnu_Lukas = 0.1998324308090579 # this is intended to mimic a physical neutrino
# density of 0.002148659574468
mnu_Matteo = 0.3
nuCDM = get_camb_Pk(model0, mnu=mnu_Lukas)
# Lukas: for the purposes of debugging, ignore the rest of these

Note: redshifts have been re-sorted (earliest first)
Note: redshifts have been re-sorted (earliest first)


In [31]:
np.save('LCDM_tiny.npy', LCDM, allow_pickle=True)
np.save('nuCDM_tiny.npy', nuCDM, allow_pickle=True)